In [1]:
import os 
import pandas as pd
import numpy as np
from pydub import AudioSegment

import IPython.display as ipd

import librosa
import librosa.display
import matplotlib.pyplot as plt

import processing
import event_manager
import epoching

pd.set_option('max_colwidth', 400)
pd.set_option('display.max_rows',100)

/Users/bastienorset/Documents/git/respiratory_sound/.venv/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
audio_folder = '../../data/ICBHI_final_database'
list_audio_files = processing.get_list_recording(audio_folder)
print('Found {} recording files'.format(len(list_audio_files)))

Found 920 recording files


In [3]:
audio_file = '222_1b1_Pr_sc_Meditron'
data_file = processing.load_file_from_recording_name(audio_folder,audio_file)
data_filtered,df_label = processing.preprocess_data(data_file,annotations='text')
data_feature= processing.apply_stft(data_filtered,n_fft=400,center=False)

In [7]:
pd.read_csv('../../data/ICBHI_final_database/222_1b1_Pr_sc_Meditron.txt',delimiter='\t',header=None)

,0,1,2,3
0,0.14197,5.2883,0,0
1,5.28830,9.5592,0,0
2,9.55920,16.8350,0,0
3,16.83500,22.2770,0,0
4,22.27700,27.3230,0,0
5,27.32300,31.4340,0,0
6,31.43400,40.1240,0,0
7,40.12400,47.4700,0,1
8,47.47000,54.4090,0,1
9,54.40900,57.7750,0,1


In [4]:
df_label

,start,end,label
0,0.14197,5.2883,0
1,5.28830,9.5592,0
2,9.55920,16.8350,0
3,16.83500,22.2770,0
4,22.27700,27.3230,0
5,27.32300,31.4340,0
6,31.43400,40.1240,0
7,40.12400,47.4700,1
8,47.47000,54.4090,1
9,54.40900,57.7750,1
